# Minimum-Cost Flow Redistricting Assignment Algorithm
## Examining An Impossibility Theoorem regarding Gerrymandering

The practice of gerrymandering, or the manipulation of electoral district boundaries for political gain, has long been a contentious issue in democratic governance. This Notebook builds upon the foundational concepts laid out in "An Impossibility Theorem for Gerrymandering" by Alexeev and Mixon, refining the established formula to enhance the analysis of electoral districting. Our research develops a sophisticated simulation model to evaluate the implications of various districting strategies on electoral fairness.

## Key parameters influencing district shapes and sizes include:
- **Delta (δ)**: Allowable deviation in population size among districts.
- **Gamma (γ)**: Standard for geographic compactness.
- **Proportion of Voters (p0)**: Ratio of voters from two major parties.
- **Grid Size (n)**: Size of the grid.
- **District Count (k)**: Number of districts.


In [1]:
import pandas as pa
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from ortools.graph import pywrapgraph
import random
import os
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, FloatSlider

### Monitor Simulation. 
#### Set to 1 if you want detailed logs

In [2]:
log_flag = 0

In [1]:
def print_info(data, name):
    if isinstance(data, list):
        print(' % ',name,' ',data,' -----type $len: ', len(data),' ', type(data),' * ')
    else:
        print(' % ',name,' ',data,' -----type ', type(data),' * ')

## Create a grid
For cross reference to Google's documentation, the census blocks are the "workers" and the centers of the districts are the "tasks."

Cost being minimized is the product of the (distance from the census block's center to the location of the center to which it is assigned by the algorithm) and (the population of the block).

Source supply is the overall population.

Capacity of each arc from the source to a census block is the population of that census block.

Capacity of each arc from a census block to a center is the population of that census block.

Capacity of each arc from a center to the sink is the ideal district size plus an allowed tolerance. How to set the tolerance is a matter of opinion. A low setting is probably preferable -- perhaps even just going with no tolerance at all.

Sink supply is the negative of the overall population.

After the algorithm is run, 0-population census blocks remain unassigned to a district because the 0 population means the "flow" across their arc is 0.

The algorithm allows a block's population to be split between districts. If this happens the block is actually assigned to the district which received the greatest share of the population by the algorithm.

V3 Update Notes:
    implements functions to calculate pp compactness and effiency gap of the districts (added voter prefernce)

In [8]:
def setup(districts,grid_size,district_centers,pops): 
    """
    Inputs: # of districts, size of grid, predeterminded district centers, total population
    Purpose: This block establishes the parameters for the situation and creates the node and edge inputs
             for the optimization procedure.
    Outputs: A tuple containing start_nodes, end_nodes, capacities, costs, supplies, source, sink, pops, total_pop
    """
    


    if log_flag == 1:
        print('in setup')
        print(' Inputs-setup \n')
        print_info(districts, 'districts')
        print_info(grid_size, 'grid_size')
        print_info(district_centers, 'district_centers')
        print_info(pops, 'pops')
        print(' end inputs \n')
    
    tol = 1 # tolerance for population difference between districts

    #Build the list holding the node labels corresponding to the sources for the graph's edges
    start_nodes = [0]*(grid_size*grid_size)   #The initial source is labeled 0 and is at the tail of edges which 
                                              #terminate at
                                              #each census block's node
    for i in range(grid_size*grid_size):
        start_nodes = start_nodes+[i+1]*districts

    for j in range(districts):
        start_nodes = start_nodes+[(grid_size*grid_size)+1+j]
        
    #Now build the list holding the node labels corresponding to the sinks for each of the graph's edges

    end_nodes=[]
    for i in range(grid_size*grid_size):
        end_nodes=end_nodes+[i+1]
    
    for j in range(grid_size*grid_size):
        for k in range(districts):
            end_nodes=end_nodes+[(grid_size*grid_size)+1+k]
        
    end_nodes=end_nodes+[(grid_size*grid_size)+districts+1]*districts

    #Now create a list of each edge's capacity.  For edges connected to census block nodes, this will be the
    #population of the census block.
    #For the edges connecting each district to the final sink node, this will be the sum of the districts'
    #populations divided by the number of districts -- i.e., the ideal district size.

    #Now use the populations passed to the function to create edge capacities.

    total_pop = np.sum(np.array(pops))
        
    capacities = np.array(pops).flatten().tolist()

    for i in np.array(pops).flatten().tolist():
        for j in range(districts):
            capacities = capacities+[i]

    for i in range(districts):
        capacities = capacities+[int(total_pop/districts)+tol]#last value here is a tolerance for how much a district
                                                              #can exceed its ideal size

    #A final set-up step involves attaching costs to each edge in the graph.
    #The costs will be the distances between each census block's centroid and the centroids of the possible
    #districts to which it could be assigned.
    #For the purposes of this example, I'm assuming census blocks have centroids located at lattice points
    #spaced 1 unit apart.
    

    #To note: edges from the initial source AND to the final sink have no cost -- and so will be given a cost of 0.

    costs = [0]*(grid_size*grid_size)

    #To calculate distances, we need to establish the locations of the district centers.

    for i in range(grid_size):  #These are the rows of the census block grid
        for j in range(grid_size):   #These are the columns of the census block grid
            for k in range(districts):
                costs = costs+[int((((i-district_centers[k][0])**2)+((j-district_centers[k][1])**2)))]

    costs = costs+[0]*districts

    #Supplies are 0 for each node except the initial source and final sink, each of which has a value 
    #corresponding to the total population to distribute.

    supplies = [int(total_pop)]+((grid_size*grid_size)+districts)*[0]+[-int(total_pop)]

    #This is the source node's label

    source = 0

    #This is the sink node's label

    sink = districts+(grid_size*grid_size)+1
    
    if log_flag == 1:
        print('Out of setup')      
        print(' output-setup \n')
        print_info(start_nodes, 'start_nodes')
        print_info(end_nodes, 'end_nodes')
        print_info(capacities, 'capacities')
        print_info(costs, 'costs')
        print_info(supplies, 'supplies')
        print_info(source, 'source')
        print_info(sink, 'sink')
        print_info(pops, 'pops')
        print_info(total_pop, 'total_pop')
        print(' end outputs \n')

    return(start_nodes, end_nodes, capacities, costs, supplies, source, sink, pops, total_pop)

### District Generation Algorithm

The districting system partitions the grid into districts based on voter locations, ensuring compliance with the one-person-one-vote principle and compactness measures. The algorithm iteratively adjusts district centers and optimizes according to the efficiency gap and compactness metrics.


In [9]:
def optimize(start_nodes, end_nodes, capacities, costs, supplies, source, sink, grid, grid_size):
    
    """
    Inputs: start_nodes, end_nodes, capacities, costs, supplies, source, sink, grid, grid_size 
    Purpose: Uses a min cost flow algorithm to optimize districting  
    Outputs: returns districting assignments
    """
    if log_flag == 1:
        print('in optimize')
    

    #Create empty DataFrame to hold outputs later
    Block_Assignments=pa.DataFrame(columns=['DIST_NO','ASSIGN_POP','ACTUAL_POP'])

    # Instantiate a SimpleMinCostFlow solver.
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

    # Add each arc.
    for i in range(len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                                capacities[i], costs[i])

    # Add node supplies.
    for i in range(len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])

    flag = 0    
    
    # Find the minimum cost flow between source and sink.
    if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
        #print('Total cost = ', min_cost_flow.OptimalCost())
        #print('\n')
        for arc in range(min_cost_flow.NumArcs()):

          # Can ignore arcs leading out of source or into sink.
          if min_cost_flow.Tail(arc)!=source and min_cost_flow.Head(arc)!=sink:
        
            if min_cost_flow.Flow(arc) > 0:
          
              #Add a way to check for those instances where a census block's population is split
              #by the algorithm and assigned to more than one block.  
              #Assign the census block to the district which optimally receives the greatest
              #fraction of its population.
            
              if min_cost_flow.Capacity(arc)==min_cost_flow.Flow(arc):
                grid.nodes[(int(((min_cost_flow.Tail(arc)-1)/grid_size)),
                        (min_cost_flow.Tail(arc)-1)%grid_size)]["district"]=min_cost_flow.Head(arc)
            
                Block_Assignments.loc[min_cost_flow.Tail(arc)]=[min_cost_flow.Head(arc),min_cost_flow.Flow(arc),
                                                            min_cost_flow.Capacity(arc)]
            
              else:
            
                if flag==0:
                    grid.nodes[(int(((min_cost_flow.Tail(arc)-1)/grid_size)),
                        (min_cost_flow.Tail(arc)-1)%grid_size)]["district"]=min_cost_flow.Head(arc)
            
                    Block_Assignments.loc[min_cost_flow.Tail(arc)]=[min_cost_flow.Head(arc),min_cost_flow.Flow(arc),
                                                            min_cost_flow.Capacity(arc)]
                
                    flag=min_cost_flow.Flow(arc)
                
                else: 
                    if Block_Assignments.loc[min_cost_flow.Tail(arc),'ASSIGN_POP']<min_cost_flow.Flow(arc):
                        grid.nodes[(int(((min_cost_flow.Tail(arc)-1)/grid_size)),
                            (min_cost_flow.Tail(arc)-1)%grid_size)]["district"]=min_cost_flow.Head(arc)
            
                        Block_Assignments.loc[min_cost_flow.Tail(arc)]=[min_cost_flow.Head(arc),min_cost_flow.Flow(arc),
                                                            min_cost_flow.Capacity(arc)]
                
                        flag = flag+min_cost_flow.Flow(arc)
                
                    else:
                        flag = flag+min_cost_flow.Flow(arc)
                    
                    if flag==min_cost_flow.Capacity(arc):
                        flag = 0
                
    else:
        print('There was an issue with the min cost flow input.')
        
    if log_flag == 1:
        print('out of optimize')

    return(Block_Assignments)

### Utitls

In [10]:
def get_pp_compactness(perimeter, area):
    """
    Inputs: perimeter and area of a districts
    Purpose: calculates ppc of districts 
    Outputs: ppc for the given districts
    """
    if log_flag == 1:
        print('in pp_compact')
        
    dist_pp_compactness = {}
    # go through a list of permeters and ares for each district
    for p_key in perimeter:
        for a_key in area:
            if p_key == a_key:
                compactness = (4 * 3.14 * int(area[a_key][0]))/(perimeter[p_key])**2
                dist_pp_compactness.setdefault(a_key,[]).append(compactness)

    if log_flag == 1:
        print('out of pp_compact')
        
    return dist_pp_compactness

In [11]:
def grid_setup(grid_s, dist, n,r,c,p_0):
    """
    Inputs: grid_size, number of districts, n, r, c, proportion of voters 0
    Purpose: sets up a grid with determined population size for each node 
    Output: districts, grid_size, district_centers, pops, grid
            a grid with population voter pref baked in 
    """
    if log_flag == 1:
        print('in grid setup')
        
    #Here we set up the grid on which the districts will be drawn
    grid_size = grid_s
    # TODO: does not need to return districts in this function
    districts = dist # number of districts
    #District centers can be specified in a variety of ways.
    #The following gives a regular horizontal spacing perturbed by a random factor, and then randomly assigns the
    #location vertically.

    district_centers = []
    spacing = int(grid_size/districts)
    for j in range(districts):
        district_centers = district_centers + [[(j+1)*spacing+random.randint(-1,1),random.randint(0,grid_size-1)]]

    #print(district_centers)

    #Now we establish the population distribution on the grid

    pops = []
    row = []
    for i in range(grid_size):
        for j in range(grid_size):
            #row = row+[i+1]                      #Use this for systematic population assignments
            #row = row+[random.randint(1,10)]     #Use this for random population assignments
            row = row+[1]                         #Use this for a population of 1 at each node
        pops = pops+[row]
        row = []

    total_pop = np.sum(np.array(pops))

    #print('Total population across all blocks is: ',total_pop)

    #And now we create the grid graph and assign the populations to the nodes

    grid = nx.grid_graph([grid_size,grid_size])
    count = 0

    for node in grid.nodes():
        grid.nodes[node]["population"]=pops[node[0]][node[1]]


    add_party_preference(grid, n, grid_size, p_0)
    
    
# to graph a figure of voters party pref spread     
#FLAG
#     plt.figure()
#     nx.draw(grid,pos= {x:x for x in grid.nodes()},node_shape='s',node_size = 20,
#             node_color = [grid.nodes[name]['voter_pref'] for name in grid.nodes()],
#             cmap='Paired')# , legend = True with_labels=True)
        
    if log_flag == 1:
        print('out of grid setup')

    return districts, grid_size, district_centers, pops, grid


In [12]:
def add_party_preference(grid, n, grid_size, p_0):
    """
    Inputs: grid of voters, n, grid size, proportion of voters a to b
    Purpose: Function adds voter party preference to each node in the graph 
    Output: No output, voter pref gets mapped to grid passed by grid_setup
    """
    if log_flag == 1:
        print('in party pref')
    squares_with_id = [] #index will serve as id
    
    all_nodes = list(grid.nodes())
    all_nodes_processed = [] #used to test zoom mechanism 
    voter_arr_ind = 0
    row_tracker = 0 # to keep track of the rows within square counting
    square = []
    voter_arr_ind = 0
    m_val_holder = 0
    vertical_mover = 0
    for i in range(n):
        if i != 0:
            vertical_mover += r*grid_size
        for m in range(n):
            row_tracker = (c*m) + vertical_mover
            #get sqaures at each level
            square = []            
            for j in range(r):
                row = all_nodes[row_tracker:row_tracker+c]
                square = square + row
                row_tracker += grid_size
            all_nodes_processed += square
            #add logic for clumping
            #if clump
            # add two squares together set squares to that
            voter_ratios = np.ones(c*r)
            #input porportions here
            proportion_of_a = math.floor((c*r)*p_0)
            voter_ratios[:proportion_of_a] = 0
            #print(voter_ratios)
            np.random.shuffle(voter_ratios)
            #print(voter_ratios)
            for node in square:    
                grid.nodes[node]['voter_pref'] = voter_ratios[voter_arr_ind] # 0 is blue 1 is brown
                voter_arr_ind = voter_arr_ind + 1

            squares_with_id.append(square)       
            voter_arr_ind = 0
            square = []
            
    ###############################
    # Set up a method for clumping (very ugly TODO...)
    ###############################
    
#     clump_randomness = random.randint(0,15)
    
#     if clump_randomness % 2 == 0:
        
#         voter_arr_ind = 0
#         row_tracker = 0 # to keep track of the rows within square counting
#         square = []
#         voter_arr_ind = 0
#         m_val_holder = 0
#         vertical_mover = 0

#         #print(squares_with_id)

#         for i in range(len(squares_with_id)):
            
#             if (i+2) == len(squares_with_id):     
#                 row_1 = squares_with_id[i]
#                 row_2 = squares_with_id[i+1]
#                 clump = row_1 + row_2
#                 #print('clump -> ' + str(clump))

#                 i+=2
#                 voter_ratios = np.ones(2*c*r)
#                 proportion_of_a = math.floor(2*(c*r)*p_0) 
#                 voter_ratios[:proportion_of_a] = 0
#                 np.random.shuffle(voter_ratios)
#                 for node in clump:    
#                     grid.nodes[node]['voter_pref'] = voter_ratios[voter_arr_ind] # 0 is blue 1 is brown
#                     voter_arr_ind = voter_arr_ind + 1
#                 voter_arr_ind = 0
#                 break

#             row_1 = squares_with_id[i]
#             row_2 = squares_with_id[i+1]
#             clump = row_1 + row_2
#             #print('clump -> ' + str(clump))

#             i+=2
#             voter_ratios = np.ones(2*c*r)
#             proportion_of_a = math.floor(2*(c*r)*p_0) 
#             voter_ratios[:proportion_of_a] = 0
#             np.random.shuffle(voter_ratios)
#             for node in clump:    
#                 grid.nodes[node]['voter_pref'] = voter_ratios[voter_arr_ind] # 0 is blue 1 is brown
#                 voter_arr_ind = voter_arr_ind + 1
#             voter_arr_ind = 0

    
    missed_nodes = list(set(all_nodes).difference(all_nodes_processed))
    if missed_nodes:   
        print("missed_nodes = " + str(missed_nodes))
        os._exit(0)
    if log_flag == 1:   
        print('out of party pref')


In [13]:
def dist_assgin(iterations, grid_size, dist_num,n,r,c,p_0):
    """
    Inputs: number iterations for assignment exploration, grid size, number of districts, n, r, c, proportion of voters a to b 
    Purpose: Here we find optimal district assignments by iterating through the optimization process, updating
             district centers by finding centroids of the current step's assignments.
    Output: districts assignemnts per itertation in a dictionary, polsby popper compactness for each iteration 
    """

    break_point = 0
    if log_flag == 1:    
        print('in dist assign')
    districts, grid_size, district_centers, pops, grid = grid_setup(grid_size,dist_num,n,r,c, p_0)
    # TODO: does not need to return districts in this function

    #Establish the number of iterations to carry out
    N = iterations
    pp_each_iter = {}
    district_data_each_iteration = {}

    for l in range(N):
        
        # if figures are produced stop iterating
        if break_point == 1:
            break
            
        state_voterPrefs_as_row = []
        # reshuffle district centers after first run
        if l!=0:
            district_centers_new=[]
            for i in range(districts):
                sumx = 0
                sumy = 0
                count = 0
                for name in grid.nodes():
                    if grid.nodes[name]['district']==i+1+(grid_size*grid_size):
                        sumx = sumx+name[0]
                        sumy = sumy+name[1]
                        count = count+1
                district_centers_new = district_centers_new+[[round(sumx/count), round(sumy/count)]]

            district_centers = district_centers_new
        # need to set up nodes once and reuse in iterations
        start_nodes, end_nodes, capacities, costs, supplies, source, sink, pops, total_pop=\
            setup(districts,grid_size,district_centers,pops)

        Block_Assignments = optimize(start_nodes, end_nodes, capacities, costs, supplies, source, sink, grid, grid_size)



        dist_area = {}
        for j in range(districts):
            pop_by_district = Block_Assignments.loc[Block_Assignments['DIST_NO']==j+1+(grid_size*grid_size),
                                                'ACTUAL_POP'].sum()
            #print()
            #district_id = grid.nodes[node]['district']
            dist_area.setdefault(j+1+(grid_size*grid_size),[]).append(pop_by_district/4)

#             print('District ',j+1+(grid_size*grid_size),' population: ',pop_by_district,
#                   '(',100*(pop_by_district-(total_pop/districts))/(total_pop/districts),'% deviation from ideal)',
#                   ' center: ',district_centers[j]) 
          

    

#         print('\n')
#         print('Ideal District Size = ',total_pop/districts,'\n\n')

#         newdir = 'Outputs_may/Maps/' +str(districts)+'/'
#         os.makedirs(os.path.dirname(newdir), exist_ok = True)

# #FLAG

# #print figure
#         #if saveMapFlag == 1:


        
        
        district_dict = {}
        edge_boundary = {}


        for node in grid.nodes():
            # create a dict with districts and assiciated nodes in them
            district_id = grid.nodes[node]['district']
            district_dict.setdefault(district_id,[]).append(node)
            # used for creating a df  for NN, row is position represents grid location, 
            # value in row represents party pref, meant to paint a picture of homogeneity for the NN.
            voter_pref = grid.nodes[node]["voter_pref"] 
            state_voterPrefs_as_row.append(voter_pref)
            #print(state_voterPrefs_as_row)
        # create a data structure to hold voting pref for each district
        win_count = {}
        a_count = 0
        b_count = 0
        party_count = 0
        for node in grid.nodes():
            district_id = grid.nodes[node]['district']
            win_count[district_id] = {}
            if party_count != 2:
                for node in grid.nodes():
                    voter_pref = grid.nodes[node]["voter_pref"]
                    win_count[district_id][voter_pref] = 0 
                    if (int(voter_pref) == 1.0) or (int(voter_pref) == 0.0):
                        party_count += 1
            if len(win_count) == dist_num:
                break;
        

        for node in grid.nodes():
            district_id = grid.nodes[node]['district']
            voter_pref = grid.nodes[node]["voter_pref"]             
            if str(voter_pref) == '1.0':
                win_count[district_id][voter_pref] += 1 
            if str(voter_pref) == '0.0':
                win_count[district_id][voter_pref] += 1 
        
        print("******")
        print(win_count.keys())
        print("!!!")
        print(win_count)
        print("********")
        
        
        win_count = find_eg(win_count)
        
        print("~~~~~~")
        print(win_count.keys())
        print("!!!")
        print(win_count)
        print("~~~~~~")
        
        
        mino_win_count = 0
        for item in win_count:
            mino_win_count = win_count[item]['win_count_0']
            break
            
        print("!!!!!!!!!!!")
    
            
        #------------------------------------------
        #save 
        # to save and quit if anomly is found 
        # print district map
        mino_win_count = 1
        if mino_win_count > 0:
            print("In save fig")
            newdir = '333distMap_n'+str(n)+'_k' +str(districts)+'Step_'+str(l)+'.png'
            #print(newdir)
            #os.makedirs(os.path.dirname(newdir), exist_ok = True)
            plt.figure()
            nx.draw(grid,pos= {x:x for x in grid.nodes()},node_shape='s',node_size = 20,
                node_color = [grid.nodes[name]['district'] for name in grid.nodes()],cmap='Paired',with_labels=False)
            plt.savefig(newdir)
            plt.close()


             # print voter pref map
            newdir1 = '333voterPrefMap_n'+str(n)+'_k' +str(districts)+'Step_'+str(l)+'.png'
            plt.figure()
            nx.draw(grid,pos= {x:x for x in grid.nodes()},node_shape='s',node_size = 20,
                node_color = [grid.nodes[name]['voter_pref'] for name in grid.nodes()],cmap='Paired',with_labels=False)
            plt.savefig(newdir1)
            plt.close()
            #break_point = 1
        print("out save fig")
            
  
        
        for key in district_dict:
            node_list = district_dict[key]
            for node in node_list:
                edge_out_of_node = grid.edges(node)
                edge_num = len(edge_out_of_node)
                if edge_num < 4:
                    edge_boundary.setdefault(key,[]).append(node)


        dist_peremeter = {}

        #Print each dist
        for key in district_dict:
            num_of_edge = 0
            District_subgraphs = district_dict[key]
            sub = grid.subgraph(District_subgraphs)
            edge_boundary.setdefault(key,[]).append(num_of_edge)
            for node in sub.nodes():
                edge_coming_out = sub.degree(node)
                if int(edge_coming_out) < 4:
                    num_of_edge = num_of_edge+1
                    dist_peremeter.update({key : num_of_edge})
# prints each district set
#             plt.figure()
#             nx.draw(sub, pos={x:x for x in sub.nodes()})


        for key in dist_area:
            win_count[key]["area"] = dist_area[key]
            
        for key in dist_area:
            win_count[key]["perimeter"] = dist_peremeter[key]

        pp_compactness = get_pp_compactness(dist_peremeter,dist_area)

        for key in pp_compactness:
            win_count[key]["pp_compactness"] = pp_compactness[key][0]
            
#         print("_____________________________----------------_-----")
#         print(win_count)
#         print("&&&&&&&&&&&&&&&")
        
#         for node in grid.nodes():
#             voter_pref = grid.nodes[node]["voter_pref"] 
#             state_voterPrefs_as_row.append(voter_pref)
        #print(win_count) 
        #for key in win_count:
        dist_key = list(win_count.keys())
        #print(dist_key)
        state_voterPrefs_as_row.append(win_count[dist_key[0]]['pOverP_a'])
        state_voterPrefs_as_row.append(win_count[dist_key[0]]['pOverP_b'])
        
     
        #print(win_count[district]['pOverP_a'])
        #state_voterPrefs_as_row.append(win_count[0][])
        
      #  print(state_voterPrefs_as_row)
#        print('len rows -> '+ str(type(state_voterPrefs_as_row)))
#        print('len cols -> '+str(len(df_cols)))


#        df_length = len(nn_df)
#        nn_df.loc[df_length] = state_voterPrefs_as_row


        #nn_df.append(state_voterPrefs_as_row)
        
      #  print(nn_df.describe)
            
        pp_each_iter.setdefault(l,[]).append(pp_compactness)
        district_data_each_iteration.setdefault(l,[]).append(win_count)
        
    if log_flag == 1: 
        print('out dist assign')
        
    return district_data_each_iteration, pp_each_iter


In [14]:
def find_eg(win_count):
    """
    Inputs: dictionary of win lose counts for parties within each district
    Purpose: find efficiency gap of district assignments
    Output: eg of districts packaged as dictionary
    """
    votes_1 = 0
    votes_0 = 0
    district_data = win_count
    total_voters = 0
    wasted_votes = []
    total_win_a = 0
    total_win_b = 0
    total_votes_state = 0
    imp_flag = 0 #set to 0 means immposiblity has not occured
    state_votes_a = 0
    state_votes_b = 0
    P_a = 0 #state_votes_a/total_votes_state
    P_b = 0
    p_a = 0 #total_win_a/state_count
    p_b = 0
    state_count = 0
#     for key in win_count:
#         district_data[key]['win_count_a'] = 0
#         district_data[key]['win_count_b'] = 0
#         district_data[key]['total_votes'] = 0
    for key in win_count:
        state_count += 1
        votes_1 = win_count[key][1.0]
        votes_0 = win_count[key][0.0]
        state_votes_a += votes_0
        state_votes_b += votes_1
        totalVotes = votes_1 + votes_0
        total_votes_state += totalVotes
        
        if votes_1 > votes_0:
            half = math.ceil(totalVotes / 2)
            wasted_1 = votes_1 - half
            imp_flag = 1
            total_win_b += 1
        else:
            wasted_1 = votes_1

        if votes_1 < votes_0:
            half = math.ceil(totalVotes / 2)
            wasted_0 = votes_0 - half
            total_win_a += 1
#             print("%%%%%%")
#             print(district_data[key]['win_count_a'])
#             print("%%%%%%")
        else:
            wasted_0 = votes_0
        
        wasted_total = wasted_0 - wasted_1
        
        wasted_votes.append(wasted_total)
        if imp_flag == 1:
            district_data[key]['imp_flag'] = 1
        else:
            district_data[key]['imp_flag'] = 0
    
    P_a =  state_votes_a/total_votes_state # vote percent
    P_b =  state_votes_b/total_votes_state
    p_a =  total_win_a/state_count # dist win percent
    p_b =  total_win_b/state_count
#     print('pa->'+str(p_a))
#     print('pb->'+str(p_b))
#     print('Pa->'+str(P_a))
#     print('Pb->'+str(P_b))
    if p_a == 0:
        pOverP_a = 0
    else:
        pOverP_a = P_a/p_a#p_a/P_a
    if p_b == 0:
        pOverP_b = 0
    else:
        pOverP_b = P_b/p_b
    sum_wv =sum(wasted_votes)
    eg = (1/total_votes_state)*sum_wv
    for key in win_count:
        district_data[key]['eg'] = eg
        district_data[key]['win_count_0'] = total_win_a
        district_data[key]['win_count_1'] = total_win_b
        district_data[key]['total_votes'] = total_votes_state
        district_data[key]['state_votes_a'] = state_votes_a
        district_data[key]['state_votes_b'] = state_votes_b
        district_data[key]['P_a'] = P_a
        district_data[key]['P_b'] = P_b
        district_data[key]['p_a'] = p_a
        district_data[key]['p_b'] = p_b
        district_data[key]['pOverP_a'] = pOverP_a
        district_data[key]['pOverP_b'] = pOverP_b
    
    return district_data 
    

In [15]:
def step_five_finder(delta, gamma, dists_num, p_0, n, c, r):
    # step 5 calculations
    delta = delta
    gamma = gamma
    k = dists_num
    proportion_a = 5
    proportion_b = (c*r) - proportion_a
    left_side = proportion_b / proportion_a
    #print("proportion a -> " + str(proportion_a) + " proportion_b -> " + str(proportion_b) + " left side -> " + str(left_side))

    F = math.sqrt((1-delta)/(2*k))

   # print('f ->'+str(F))
    numerator = (F**2)-(4*3.14*gamma**(-1)*F*math.sqrt(2)*(1/n))-(8*((3.14)**2)*(gamma**(-1))*(1/n)**2)
   # print('num ->' +str(numerator))
    denominator = (F**2)+(4*3.14*gamma**(-1)*F*math.sqrt(2)*(1/n))+(8*((3.14)**2)*(gamma**(-1))*(1/n)**2)
   # print('dem ->' +str(denominator))
    right_side = numerator/denominator
    
    return left_side, right_side

In [16]:
def refined_step_five_finder(delta, gamma, dists_num, p_0, n, c, r):
    # step 5 calculations
    delta = delta
    gamma = gamma
    k = dists_num
    proportion_a = 5
    proportion_b = (c*r) - proportion_a
    left_side = proportion_b / proportion_a
    #print("proportion a -> " + str(proportion_a) + " proportion_b -> " + str(proportion_b) + " left side -> " + str(left_side))

    F = math.sqrt((1-delta)/(2*k))

   # print('f ->'+str(F))
    numerator = 2*(1/n)*(math.pi)
   # print('num ->' +str(numerator))
    denominator = (4*gamma)*(math.sqrt((1-delta)/(k+1)))*n
   # print('dem ->' +str(denominator))
    right_side = numerator/denominator
    
    return left_side, right_side

In [17]:
def find_winners(win_count):
    partyOneWin = 0
    partyZeroWin = 0
    for iteration in win_count:
        for districts in win_count[iteration]:
            for district in districts:
                for party in districts[district]:

                    if party == 1.0 :
                        if str(districts[district][party]) == '{}':
                            partyOneCount = 0
                        else:
                            partyOneCount = int(str(districts[district][party]))
#                         print("in 1")
#                         print(partyOneCount)
                    if party == 0.0:
                        partyZeroCount = int(str(districts[district][party]))
#                         print("in 0")
#                         print(partyZeroCount)

                if partyOneCount > partyZeroWin:
                    partyOneCount += 1
                if partyOneCount < partyZeroWin:
                    partyZeroCount += 1
    return partyOneWin, partyZeroWin

In [18]:
def create_dataFrame(win_count, extra):
    data = []
    row = []
    votesA = 0
    votesB = 0
    area = 0
    perimeter = 0
    pp_compactness = 0
    eg = 0
    imp_flag = 0
    win_count_a = 0
    win_count_b = 0
    for iteration in win_count:
        for district in win_count[iteration][0]:
            #print(district)
            #print(win_count[iteration][0][district])
            for key in win_count[iteration][0][district]:
                #print(win_count[iteration][0][district][key])
                if key == 0.0:
                    votesA = win_count[iteration][0][district][key]
                if key == 1.0:
                    votesB = win_count[iteration][0][district][key]
                if key == 'area':
                    area = win_count[iteration][0][district][key][0]
                if key == 'perimeter':
                    perimeter = win_count[iteration][0][district][key]
                if key == 'pp_compactness':
                    pp_compactness = win_count[iteration][0][district][key]
                if key == 'eg':
                    eg = win_count[iteration][0][district][key]
                if key == 'imp_flag':
                    imp_flag = win_count[iteration][0][district][key]
                if key == 'win_count_0':
                    win_count_a = win_count[iteration][0][district][key]
                if key == 'win_count_1':
                    win_count_b = win_count[iteration][0][district][key]
                if key == 'total_votes':
                    total_votes = win_count[iteration][0][district][key]
                if key == 'state_votes_a':
                    state_votes_a = win_count[iteration][0][district][key]
                if key == 'state_votes_b':
                    state_votes_b = win_count[iteration][0][district][key]
                if key == 'P_a':
                    P_a = win_count[iteration][0][district][key]                    
                if key == 'P_b':
                    P_b = win_count[iteration][0][district][key]                  
                if key == 'p_a':
                    p_a = win_count[iteration][0][district][key]   
                if key == 'p_b':
                    p_b = win_count[iteration][0][district][key]   
                if key == 'pOverP_a':
                    pOverP_a = win_count[iteration][0][district][key]   
                if key == 'pOverP_b':
                    pOverP_b = win_count[iteration][0][district][key] 

        
        
            #wastedA, wastedB = find_wasted_votes(votesA, votesB)
            row.append(votesA)
            row.append(votesB)
            row.append(area)
            row.append(perimeter)
            row.append(pp_compactness)
            row.append(eg)
            row.append(imp_flag)
            row.append(win_count_a)
            row.append(win_count_b)
            row.append(total_votes)
            row.append(state_votes_a)
            row.append(state_votes_b)   
            row.append(P_a)
            row.append(P_b)
            row.append(p_a)
            row.append(p_b)
            row.append(pOverP_a)
            row.append(pOverP_b)
            for item in extra:
                row.append(item)
            data.append(row)
            row = []

    #print(data)
    df = pa.DataFrame(data, columns = ['votesA', 'votesB', 'area', 'perimeter', 'pp_compactness', 'eg', 'imp_flag', 'win_count_a', 'win_count_b', 'total_votes','state_votes_a','state_votes_b','P_a','P_b','p_a','p_b','pOverP_a','pOverP_b', 'n', 'L', 'K', 'proportion_a',  'left_side', 'right_side', 'refined_left_side', 'refinded_right_side'])
    #print(df)
    return df
      

In [19]:
main = pa.DataFrame(columns=['votesA', 'votesB', 'area', 'perimeter','pp_compactness', 'eg', 'imp_flag', 'win_count_a', 'win_count_b',  'total_votes','state_votes_a','state_votes_b','P_a','P_b','p_a','p_b','pOverP_a','pOverP_b','n', 'L', 'K', 'proportion_a',  'left_side', 'right_side', 'refined_left_side', 'refinded_right_side'])


In [37]:
# # define constants N5K6
# partyOneWin = 0
# partyZeroWin = 0
# p_0 = 0.555 # prop of a - proportion of a to b value range 0-1
# # define K
# dists_num = 6 #starting K
# #define Square detials
# n = 5 # num of sqs in a a row
# c = 3 # col count within sq
# r = 3 # row count within sq
# grid_length = n*c #entire grid length
# #locked in variables (for now!)
# delta = 0.05
# gamma = 0.2
# iterations = 1000
# counterrr = 0

# for n in range(6,21):
#     for k in range(2,11):
#         # define constants N5K6
#         partyOneWin = 0
#         partyZeroWin = 0
#         p_0 = 0.555 # prop of a - proportion of a to b value range 0-1
#         dists_num = k 
#         c = 3
#         r = 3
#         grid_length = n*c #entire grid length
#         #locked in variables (for now!)
#         delta = 0.05
#         gamma = 0.2
#         iterations = 1
#         left_side, right_side = step_five_finder(delta, gamma, dists_num, p_0, n, c, r)
#         refined_left_side, refined_right_side = refined_step_five_finder(delta, gamma, dists_num, p_0, n, c, r)
#         each_dist_num_results = {}
#         win_count, pp_each_iter = dist_assgin(iterations,grid_length,dists_num,n,r,c,p_0)
#         partyOneWin, partyZeroWin = find_winners(win_count)
#         extra = [n, (c*r), dists_num, p_0, left_side, right_side, refined_left_side, refined_right_side]
#         df = create_dataFrame(win_count, extra)
#         main = pa.concat([df, main], ignore_index=True)
#         print(main)
#         break
#     break

In [38]:
# csvDir = 'May_output_'+str(n)+'_'+str(dists_num)+'.csv'
# main.to_csv(csvDir,index=False)
# main.describe()
# print("Done")